In [20]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import box
import numpy as np
from shapely import wkt
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point
from pyproj import Proj, transform, CRS
from folium.plugins import MarkerCluster

In [2]:
import matplotlib.font_manager as fm

# 한국어 출력을 위한 폰트 설정
plt.rc('font', family='NanumGothic') 

# 마이너스 기호가 깨지는 것을 방지
plt.rcParams['axes.unicode_minus'] = False

In [3]:
df = pd.read_csv("서울시_비상소화장치_좌표.csv")
gdf = gpd.read_file("data/동경계/동경계_geo.shp")

In [4]:
df

,연번,비상소화장치일련번호,비상소화장치고유번호,geometry
0,1,110080030,41,POINT (126.9143790416038 37.570542820324434)
1,2,110080031,54,POINT (126.90910983392715 37.588984348035225)
2,3,110080032,56,POINT (126.95137336075969 37.596039805648054)
3,4,110080033,57,POINT (126.9506986396432 37.600517642130285)
4,5,110080034,55,POINT (126.90423968355844 37.5960208309644)
...,...,...,...,...
2690,2691,110086927,000118,POINT (126.95322756859754 37.5568313452144)
2691,2692,110086928,000119,POINT (126.95448651039091 37.55682861400999)
2692,2693,110086929,000120,POINT (126.95530769419845 37.55684582034688)
2693,2694,110086930,000121,POINT (126.95506274700165 37.55627095344716)


In [5]:
gdf

,구,동,geometry
0,종로구,청운동,"POLYGON ((126.97556 37.58968, 126.97549 37.589..."
1,종로구,신교동,"POLYGON ((126.97031 37.58418, 126.97033 37.584..."
2,종로구,궁정동,"POLYGON ((126.97400 37.58654, 126.97401 37.586..."
3,종로구,효자동,"POLYGON ((126.97356 37.58323, 126.97355 37.582..."
4,종로구,창성동,"POLYGON ((126.97353 37.58182, 126.97354 37.581..."
...,...,...,...
462,강동구,둔촌동,"POLYGON ((127.15669 37.53756, 127.15654 37.537..."
463,강동구,암사동,"POLYGON ((127.14353 37.55490, 127.14343 37.554..."
464,강동구,성내동,"POLYGON ((127.13424 37.53556, 127.13423 37.535..."
465,강동구,천호동,"POLYGON ((127.14153 37.55080, 127.14154 37.550..."


In [13]:
df_gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [15]:
joined_gdf = gpd.sjoin(df_gdf, gdf, how="inner", predicate='intersects')

C:\Users\1qlqj\AppData\Local\Temp\ipykernel_33936\2070018927.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined_gdf = gpd.sjoin(df_gdf, gdf, how="inner", predicate='intersects')


In [16]:
joined_gdf

,연번,비상소화장치일련번호,비상소화장치고유번호,geometry,index_right,구,동
0,1,110080030,41,POINT (126.91438 37.57054),319,서대문구,남가좌동
1,2,110080031,54,POINT (126.90911 37.58898),298,은평구,증산동
2,3,110080032,56,POINT (126.95137 37.59604),317,서대문구,홍은동
3,4,110080033,57,POINT (126.95070 37.60052),317,서대문구,홍은동
4,5,110080034,55,POINT (126.90424 37.59602),297,은평구,신사동
...,...,...,...,...,...,...,...
2690,2691,110086927,000118,POINT (126.95323 37.55683),320,마포구,아현동
2691,2692,110086928,000119,POINT (126.95449 37.55683),320,마포구,아현동
2692,2693,110086929,000120,POINT (126.95531 37.55685),320,마포구,아현동
2693,2694,110086930,000121,POINT (126.95506 37.55627),320,마포구,아현동


In [17]:
joined_gdf = joined_gdf.drop(columns = ['index_right'])

In [19]:
joined_gdf

,연번,비상소화장치일련번호,비상소화장치고유번호,geometry,구,동
0,1,110080030,41,POINT (126.91438 37.57054),서대문구,남가좌동
1,2,110080031,54,POINT (126.90911 37.58898),은평구,증산동
2,3,110080032,56,POINT (126.95137 37.59604),서대문구,홍은동
3,4,110080033,57,POINT (126.95070 37.60052),서대문구,홍은동
4,5,110080034,55,POINT (126.90424 37.59602),은평구,신사동
...,...,...,...,...,...,...
2690,2691,110086927,000118,POINT (126.95323 37.55683),마포구,아현동
2691,2692,110086928,000119,POINT (126.95449 37.55683),마포구,아현동
2692,2693,110086929,000120,POINT (126.95531 37.55685),마포구,아현동
2693,2694,110086930,000121,POINT (126.95506 37.55627),마포구,아현동


In [26]:
joined_gdf.to_csv("data/서울시_비상소화장치_좌표_구동.csv")